In [ ]:
import pandas as pd
from openff.toolkit.topology import Molecule, Topology
from openff.toolkit.typing.engines.smirnoff import ForceField
from openff.units import unit

from openff.interchange import Interchange
from openff.interchange.drivers import get_gromacs_energies, get_openmm_energies

In [ ]:
SMILES = 10 * "C"  #  "c1n(CCO)c(C(F)(F)(F))cc1CNCCl"

In [ ]:
molecule = Molecule.from_smiles(SMILES)
molecule.generate_conformers(n_conformers=20, rms_cutoff=0.1 * unit.angstrom),
topology = molecule.to_topology()

In [ ]:
sage = ForceField("openff-2.0.0.offxml")

In [ ]:
interchange = Interchange.from_smirnoff(force_field=sage, topology=topology)

In [ ]:
interchange.positions = molecule.conformers[0]
interchange.box = unit.Quantity([4, 4, 4], unit.nanometer)

system = interchange.to_openmm()

In [ ]:
summary = pd.DataFrame()

kj_mol = unit.kilojoule / unit.mol

for idx, conformer in enumerate(molecule.conformers):
    interchange.positions = conformer

    openmm_energies = get_openmm_energies(interchange).total_energy.m_as(kj_mol)
    gromacs_energies = get_gromacs_energies(interchange).total_energy.m_as(kj_mol)

    summary = pd.concat(
        [
            summary,
            pd.DataFrame.from_dict(
                {
                    "Conformer No.": [idx],
                    "Interchange -> OpenMM (kJ/mol)": [round(openmm_energies, 3)],
                    "Interchange -> GROMACS": [round(gromacs_energies, 3)],
                }
            ),
        ]
    )

In [ ]:
summary.style.hide(axis="index")